In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,63.28,75,99,4.92,overcast clouds
1,1,Namibe,AO,-15.1961,12.1522,75.13,84,100,8.75,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,74.03,86,100,8.32,moderate rain
3,3,Isla Aguada,MX,18.7833,-91.4667,75.38,86,66,12.84,broken clouds
4,4,Mareeba,AU,-17.0000,145.4333,88.66,71,26,3.49,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp= float(input("What is your maximum temperature criteria?"))
min_temp= float(input("What is your minimum temperature criteria?"))

What is your maximum temperature criteria?90
What is your minimum temperature criteria?70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df= city_data_df.loc[(city_data_df['Max Temp']>= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Namibe,AO,-15.1961,12.1522,75.13,84,100,8.75,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,74.03,86,100,8.32,moderate rain
3,3,Isla Aguada,MX,18.7833,-91.4667,75.38,86,66,12.84,broken clouds
4,4,Mareeba,AU,-17.0000,145.4333,88.66,71,26,3.49,scattered clouds
5,5,Koumac,NC,-20.5667,164.2833,86.77,59,22,2.46,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Namibe,AO,75.13,overcast clouds,-15.1961,12.1522,
2,Rikitea,PF,74.03,moderate rain,-23.1203,-134.9692,
3,Isla Aguada,MX,75.38,broken clouds,18.7833,-91.4667,
4,Mareeba,AU,88.66,scattered clouds,-17.0000,145.4333,
5,Koumac,NC,86.77,few clouds,-20.5667,164.2833,
6,Mataura,NZ,74.48,clear sky,-46.1927,168.8643,
7,Avarua,CK,80.65,light rain,-21.2078,-159.7750,
14,Vaini,TO,81.63,few clouds,-21.2000,-175.2000,
16,Meulaboh,ID,79.93,overcast clouds,4.1363,96.1285,
17,Cayenne,GF,76.08,moderate rain,4.9333,-52.3333,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Places API to get JSON data.
    base_url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params= params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Iterate through the DataFrame.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Namibe,AO,75.13,overcast clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
2,Rikitea,PF,74.03,moderate rain,-23.1203,-134.9692,People ThankYou
3,Isla Aguada,MX,75.38,broken clouds,18.7833,-91.4667,Freedom Shores
4,Mareeba,AU,88.66,scattered clouds,-17.0000,145.4333,Trinity Plains Tourist Park
5,Koumac,NC,86.77,few clouds,-20.5667,164.2833,Le Monitel
6,Mataura,NZ,74.48,clear sky,-46.1927,168.8643,Ellie's Villa
7,Avarua,CK,80.65,light rain,-21.2078,-159.7750,Paradise Inn
14,Vaini,TO,81.63,few clouds,-21.2000,-175.2000,Keleti Beach Resort
16,Meulaboh,ID,79.93,overcast clouds,4.1363,96.1285,Kost Pria
17,Cayenne,GF,76.08,moderate rain,4.9333,-52.3333,Hôtel Le Dronmi


In [14]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description </dt><dd>{Current Description}</dd><dt><dd> and <dd><dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

SyntaxError: invalid syntax (2657661369.py, line 1)